 we will discover how to develop a CNN to classify images of dogs and cats using a pre-trained dataset VGG-16.It is Binary Classification.

Dataset : The train folder contains 25,000 images of dogs and cats. Each image in this folder has the label as part of the filename. The test folder contains 12,500 images, named according to a numeric id. For each image in the test set, you should predict a probability that the image is a dog (1 = dog, 0 = cat).

Method : For the solution of this problem we will use a pre-trained model,VGG16.

In [ ]:
#2. Import necessary libraries

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.models import Model

In [29]:
#Load the Pretrained model
#VGG16

model = tf.keras.applications.vgg16.VGG16()

In [30]:
# Extract the Zip File ---- Step required in colab if zip file uploaded

!unzip /content/cats_and_dogs.zip


Archive:  /content/cats_and_dogs.zip
replace cats_and_dogs/train/cats/cat.0.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [31]:
# Preprocessing --- Goal is to make the data compatible for CNN
# Tensorflow by default offers direct class to achieve preprocessing

# ImageGenerators

train_generator = tf.keras.preprocessing.image.ImageDataGenerator(preprocessing_function=tf.keras.applications.vgg16.preprocess_input)
test_generator = tf.keras.preprocessing.image.ImageDataGenerator(preprocessing_function=tf.keras.applications.vgg16.preprocess_input)

In [32]:
# Pass my images

trainImageData = train_generator.flow_from_directory('/content/cats_and_dogs',
                                                     batch_size=20, #How many images to pass per tick
                                                     class_mode='categorical', #binary --- Binary Classification | categorical ---- Multi-class classification
                                                     target_size=(224,224))

testImageData = test_generator.flow_from_directory('/content/cats_and_dogs',
                                                     batch_size=20, #How many images to pass per tick
                                                     class_mode='categorical', #binary --- Binary Classification | categorical ---- Multi-class classification
                                                     target_size=(224,224))

Found 3000 images belonging to 2 classes.
Found 3000 images belonging to 2 classes.


In [33]:
# Reuse the VGG16 architecture

#include_top=False ------------------> Removing the FC layer of the architecture

vgg1 = tf.keras.applications.vgg16.VGG16(weights="imagenet", include_top=False)

In [34]:
#lock the weights of Vgg convolution layer

for layer in vgg1.layers:
  layer.trainable=False

In [35]:
#==================NOTE: Only for New Version of tf (tf >= 2.15)================================
# Define the model with an explicit Input layer
input_layer = tf.keras.layers.Input(shape=[224, 224, 3])
x = vgg1(input_layer)
x = tf.keras.layers.Flatten()(x)
x = tf.keras.layers.Dense(4000, activation="relu")(x)
x = tf.keras.layers.Dense(2000, activation="relu")(x)
output_layer = tf.keras.layers.Dense(2, activation="softmax")(x)

# Create the final model
model = tf.keras.models.Model(inputs=input_layer, outputs=output_layer)

In [36]:
model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_5 (InputLayer)      │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ vgg16 (Functional)              │ (None, 7, 7, 512)      │    14,714,688 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 25088)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 4000)           │   100,356,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 2000)           │     8,002,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 2)              │         4,002 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 123,076,690 (469.50 MB)

 Trainable params: 108,362,002 (413.37 MB)

 Non-trainable params: 14,714,688 (56.13 MB)

In [37]:
#Compile
model.compile(optimizer="adam",
              loss="categorical_crossentropy",
              metrics=['accuracy'])

In [ ]:
model.fit(trainImageData,
          validation_data=testImageData,
          epochs=15,
          steps_per_epoch=int(len(trainImageData.filenames) / trainImageData.batch_size),
          validation_steps=int(len(testImageData.filenames) / testImageData.batch_size))

Epoch 1/15


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


150/150 ━━━━━━━━━━━━━━━━━━━━ 44s 276ms/step - accuracy: 0.5512 - loss: 48.6170 - val_accuracy: 0.6763 - val_loss: 2.2599
Epoch 2/15
150/150 ━━━━━━━━━━━━━━━━━━━━ 81s 276ms/step - accuracy: 0.6747 - loss: 1.2849 - val_accuracy: 0.8300 - val_loss: 0.4626
Epoch 3/15
150/150 ━━━━━━━━━━━━━━━━━━━━ 81s 272ms/step - accuracy: 0.8474 - loss: 0.4076 - val_accuracy: 0.8800 - val_loss: 0.3958
Epoch 4/15
150/150 ━━━━━━━━━━━━━━━━━━━━ 40s 270ms/step - accuracy: 0.9427 - loss: 0.1709 - val_accuracy: 0.9750 - val_loss: 0.0702
Epoch 5/15
150/150 ━━━━━━━━━━━━━━━━━━━━ 0s 142ms/step - accuracy: 0.9812 - loss: 0.0726

In [ ]:
import tensorflow as tf
import numpy as np

# Load the image and preprocess it
img = tf.keras.preprocessing.image.load_img('cat.jpg', target_size=(224, 224))  # Replace 'cat.jpg' with the actual path
img_array = tf.keras.preprocessing.image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)

# Make the prediction
prediction = model.predict(img_array)

# Get the predicted class label
class_labels = {v: k for k, v in trainImageData.class_indices.items()}
predicted_class_index = np.argmax(prediction)
predicted_class_label = class_labels[predicted_class_index]

# Print the prediction
print(predicted_class_label)


In [ ]:
img = tf.keras.preprocessing.image.load_img('cat.jpg', target_size=(224, 224))

In [ ]:
import matplotlib.pyplot as plt
plt.imshow(img)

In [ ]:
imgArray= tf.keras.preprocessing.image.img_to_array(img)
imgArray.shape

In [ ]:
import numpy as np
compatibleArray = np.expand_dims(imgArray, axis=0)
compatibleArray.shape

In [ ]:
prediction=model.predict(compatibleArray)

In [ ]:
trainImageData.class_indices.items()

In [ ]:
{v:k for k,v in trainImageData.class_indices.items()} [ int(prediction[0][0]) ]

In [ ]:
predicted_class_index = np.argmax(model.predict(compatibleArray))
class_labels = {v: k for k, v in trainImageData.class_indices.items()}
predicted_class_label = class_labels[predicted_class_index]
print(predicted_class_label)